In [2]:
import pandas as pd
import numpy as np
import requests
from time import sleep
import random 
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import math

In [145]:
years = np.arange(2016,2019,1)

In [146]:
# https://www.nytimes.com/search?endDate={year}1231&query=immigration%20%20U.S.&sort=best&startDate={year}0101
def num_of_pages(year):
    url = f'https://www.nytimes.com/search?endDate={year}1231&query=immigration%20%20U.S.&sort=best&startDate={year}0101'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    search_page = response.text
    soup = BeautifulSoup(search_page,"html5lib")
    search_result = soup.find_all(class_='SearchForm-searchStatus--2Z3Tw')
    search_result_str = str(search_result)
    num_of_articles = re.match(r'.*Showing\s(\d,*\d+)\sresults.*',search_result_str).group(1)
    num = int(num_of_articles.replace(',',''))
    return(num)

In [147]:
news_per_year = defaultdict()
for year in years:
    news_per_year[year]=num_of_pages(year)
    print(year, ': ', num_of_pages(year))

2016 :  1776
2017 :  1574
2018 :  824


In [148]:
API_ROOT = 'https://api.nytimes.com/svc/search/v2/articlesearch.json'

API_SIGNUP_PAGE = 'http://developer.nytimes.com/docs/reference/keys'

In [150]:
#keys = ['be9fb4178ca0420ca4940f297693bc50']
#keys= ['fbdb0bf2114c49a4af9d5a810f5ad0d5']
#keys = ['0ece8772d2304360922e4d635d1b1d03']
keys = ['70ea6a4cbd0a4ad088b1f316e66afbd2']

In [151]:
def create_request(start_date, end_date, page_num):
    s = len(keys)-1
    API_KEY = keys[random.randint(0,s)]
    resp = requests.get(API_ROOT, params={
        'api-key': API_KEY,
        'q': "immigration U.S.",
        'begin_date': start_date,
        'end_date': end_date,
        'sort': "oldest",
        'page': page_num})

    print(resp)
    return resp
#     return(resp.json())

In [152]:
def parse_search_res(json_file):
    news = []
    for i in json_file['response']['docs']:   
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['date'] = i['pub_date'][0:10]
        dic['score'] = i['score']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        if 'source' in i:
            dic['source'] = i['source']
        if 'type_of_material' in i:
            dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        news.append(dic)
    return(news)

In [153]:
math.ceil(142/10)+1

16

In [154]:
for k, v in news_per_year.items():
    data = []
    start_date = f'{k}0101'
    end_date = f'{k}1231'
    pages = math.ceil(v/10)+1
    sleep(2)
    for page in range(pages):
        resp = create_request(start_date,end_date,page)
        if resp.ok == True:
            data.extend(parse_search_res(resp.json()))
        else:
            print((page, resp.text))
#         data.extend(parse_search_res(create_request(start_date,end_date,page)))
        sleep(2)
    df = pd.DataFrame.from_dict(data)
    df.to_csv(f'nytimes_meta_{k}.csv')
    print('Collected:', k)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200